# Main Script

STATIC VERSION
- Given a fixed DoD
- Main function
- Auxillary functions:
    - initial solution creator: fill up vehicles while respecting capacity
    - (greedy) insertion heuristic: in stead of looking for all possible insertions, look for a better insertion until you find no better one
        - initial solution creator > spatio-temporal clustering of requests?
        - feasibility check
        - incremental waiting time + travel time calculator
    - improvement heuristic
        - neighborhood creators (destroy&repair, swappers, L-opt moves, ...)
        - evaluate objective functions

## Todo (update 16/2)

- Test basic network loading functions for larger instances! 
- Some vehicles can drive all the way to 5 (1 > 5 > 1), others return from 3 directly back (1 > 3 > 1)
    - make it that some vehicles are available at the depot: these can be schedules for new rides

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

import network_generation as netg
import requests_generation as rg
import solution_generation as sg
import solution_evaluation as se
import solution_visualisation as sv

In [2]:
# PARAMETERS

#network
network_size = 'small'
interstop_distance = 'half'
v_mean = 50 #km/h

#requests
demand_scenario = 2
time_of_day = 1 #1 = peak, 0 = off-peak
peak_duration = 60 #min.
req_max_cluster_time = 5 #min.

#vehicles
available_vehicles = 36
vehicle_capacity = 20
dod = 0   #static version = 0, dynamic version > 0

#objective function
WT_pen = 3 
TT_pen = 1 

## Load network

In [3]:
# network
network = netg.import_network(network_size, interstop_distance)
od_matrix = netg.generate_tt_od(network, v_mean)
od_matrix

{(1, 1): 0.0,
 (1, 2): 5.9687184554140265,
 (1, 3): 6.0,
 (1, 4): 5.9687184554140265,
 (1, 5): 0.0,
 (2, 1): 5.9687184554140265,
 (2, 2): 0.0,
 (2, 3): 5.9687184554140265,
 (2, 4): 0.0,
 (2, 5): 5.9687184554140265,
 (3, 1): 6.0,
 (3, 2): 5.9687184554140265,
 (3, 3): 0.0,
 (3, 4): 5.9687184554140265,
 (3, 5): 6.0,
 (4, 1): 5.9687184554140265,
 (4, 2): 0.0,
 (4, 3): 5.9687184554140265,
 (4, 4): 0.0,
 (4, 5): 5.9687184554140265,
 (5, 1): 0.0,
 (5, 2): 5.9687184554140265,
 (5, 3): 6.0,
 (5, 4): 5.9687184554140265,
 (5, 5): 0.0}

## Import requests

In [4]:
lambdapeak = rg.get_scenario_mean_demand('city', network_size, scen=demand_scenario, peak=1)
mupeak = rg.get_scenario_mean_demand('terminal', network_size, scen=demand_scenario, peak=1)

In [5]:
dict_requests = rg.convert_md_todict(lambdapeak, mupeak, demand_scenario)
total_requests = rg.generate_requests(dict_requests, peak_duration)
od_pairs = set(total_requests.keys())

In [6]:
list_all_requests = rg.list_all_requests(total_requests)
list_all_requests

[((1, 2), 0, 0),
 ((1, 2), 7.9587450816311005, 0),
 ((1, 2), 15.917490163262201, 0),
 ((1, 2), 23.8762352448933, 0),
 ((1, 2), 31.834980326524402, 0),
 ((1, 2), 39.7937254081555, 0),
 ((1, 2), 47.7524704897866, 0),
 ((1, 2), 55.7112155714177, 0),
 ((1, 3), 0, 0),
 ((1, 3), 0.1989686270407775, 0),
 ((1, 3), 0.397937254081555, 0),
 ((1, 3), 0.5969058811223324, 0),
 ((1, 3), 0.79587450816311, 0),
 ((1, 3), 0.9948431352038876, 0),
 ((1, 3), 1.1938117622446651, 0),
 ((1, 3), 1.3927803892854427, 0),
 ((1, 3), 1.5917490163262202, 0),
 ((1, 3), 1.7907176433669978, 0),
 ((1, 3), 1.9896862704077753, 0),
 ((1, 3), 2.188654897448553, 0),
 ((1, 3), 2.3876235244893302, 0),
 ((1, 3), 2.5865921515301076, 0),
 ((1, 3), 2.785560778570885, 0),
 ((1, 3), 2.9845294056116622, 0),
 ((1, 3), 3.1834980326524396, 0),
 ((1, 3), 3.382466659693217, 0),
 ((1, 3), 3.5814352867339942, 0),
 ((1, 3), 3.7804039137747716, 0),
 ((1, 3), 3.979372540815549, 0),
 ((1, 3), 4.178341167856327, 0),
 ((1, 3), 4.377309794897104, 0

In [7]:
# requests are grouped per 5 minutes from the 1st departure of a group
copy_list_requests = list_all_requests.copy()
grouped_requests = rg.group_requests_dt(copy_list_requests, req_max_cluster_time, od_pairs)
grouped_requests

{(2,
  3): [[((2, 3), 0, 0),
   ((2, 3), 0.1989686270407775, 0),
   ((2, 3), 0.397937254081555, 0),
   ((2, 3), 0.5969058811223324, 0),
   ((2, 3), 0.79587450816311, 0),
   ((2, 3), 0.9948431352038876, 0),
   ((2, 3), 1.1938117622446651, 0),
   ((2, 3), 1.3927803892854427, 0),
   ((2, 3), 1.5917490163262202, 0),
   ((2, 3), 1.7907176433669978, 0),
   ((2, 3), 1.9896862704077753, 0),
   ((2, 3), 2.188654897448553, 0),
   ((2, 3), 2.3876235244893302, 0),
   ((2, 3), 2.5865921515301076, 0),
   ((2, 3), 2.785560778570885, 0),
   ((2, 3), 2.9845294056116622, 0),
   ((2, 3), 3.1834980326524396, 0),
   ((2, 3), 3.382466659693217, 0),
   ((2, 3), 3.5814352867339942, 0),
   ((2, 3), 3.7804039137747716, 0),
   ((2, 3), 3.979372540815549, 0),
   ((2, 3), 4.178341167856327, 0),
   ((2, 3), 4.377309794897104, 0),
   ((2, 3), 4.576278421937881, 0),
   ((2, 3), 4.775247048978659, 0),
   ((2, 3), 4.974215676019436, 0)], [((2, 3), 5.173184303060213, 0),
   ((2, 3), 5.372152930100991, 0),
   ((2, 3), 5.

In [8]:
# this shows how big the groups of passengers are (per threshold time from the first departure) + how many there are
count_groups = rg.request_groups_per_od(grouped_requests)
count_groups

{(2, 3): [26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 16],
 (1, 3): [26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 16],
 (4, 5): [1, 1],
 (1, 2): [1, 1, 1, 1, 1, 1, 1, 1],
 (3, 5): [1, 1, 1, 1, 1, 1, 1, 1],
 (3, 4): [1, 1, 1, 1, 1, 1, 1, 1]}

## Initial solution generation

Insertion strategy: momentarily focussed on the capacity constraint (i.e. trying to fill vehicles), not so much on the time constraint

1. Divide requests into chunks: 
 * (a) of at most equal size (e.g. 10) 
 * (b) group requests as long as the maximum waiting time is e.g. 5 min.
2. Select a first chunk of travellers, who want to travel from the origin to the farthest point, and fill a bus with them (almost) 
    * If this bus is ride is full, then this schedule is complete.
    * If there is room left, then add passengers in between at each visited stop.


In [9]:
terminal, city, terminal_end = netg.get_network_boundaries(network)
network_dim = terminal, city, terminal_end
# find a more systematic way to decide which services make the entire trip
round_trips = {1, 2, 3, 4, 5, 6, 7, 8, 9}

first_veh_index = 1

copy_grouped_requests = grouped_requests.copy()

In [10]:
initial = sg.create_initial_solution(copy_grouped_requests, city, terminal_end, network_dim, first_veh_index,  
                                     available_vehicles, round_trips, vehicle_capacity)

In [11]:
corr_initial = sg.correct_dep_times(initial, od_matrix, round_trips, network_dim)
corr_initial

{1: {1: [1.7907176433669978,
   [((1, 3), 0, 0),
    ((1, 3), 0.1989686270407775, 0),
    ((1, 3), 0.397937254081555, 0),
    ((1, 3), 0.5969058811223324, 0),
    ((1, 3), 0.79587450816311, 0),
    ((1, 3), 0.9948431352038876, 0),
    ((1, 3), 1.1938117622446651, 0),
    ((1, 3), 1.3927803892854427, 0),
    ((1, 3), 1.5917490163262202, 0),
    ((1, 3), 1.7907176433669978, 0)],
   [((1, 2), 0, 0)]],
  2: [7.759436098781024,
   [((1, 3), 0, 0),
    ((1, 3), 0.1989686270407775, 0),
    ((1, 3), 0.397937254081555, 0),
    ((1, 3), 0.5969058811223324, 0),
    ((1, 3), 0.79587450816311, 0),
    ((1, 3), 0.9948431352038876, 0),
    ((1, 3), 1.1938117622446651, 0),
    ((1, 3), 1.3927803892854427, 0),
    ((1, 3), 1.5917490163262202, 0),
    ((1, 3), 1.7907176433669978, 0)],
   [((2, 3), 0, 0),
    ((2, 3), 0.1989686270407775, 0),
    ((2, 3), 0.397937254081555, 0),
    ((2, 3), 0.5969058811223324, 0),
    ((2, 3), 0.79587450816311, 0),
    ((2, 3), 0.9948431352038876, 0),
    ((2, 3), 1.19381

In [12]:
copy_grouped_requests

{(2, 3): [[], [], [], [], [], [], [], [], [], [], [], []],
 (1, 3): [[], [], [], [], [], [], [], [], [], [], [], []],
 (4, 5): [[], []],
 (1, 2): [[], [], [], [], [], [], [], []],
 (3, 5): [[], [], [], [], [], [], [], []],
 (3, 4): [[], [], [], [], [], [], [], []]}

## Evaluate solution

In [13]:
waiting_time_dict = se.calc_waiting_time(corr_initial)
inveh_time_dict = se.calc_in_vehicle_time(corr_initial)
total_tt_dict = se.calculate_ttt(inveh_time_dict, waiting_time_dict)

In [14]:
sum_stops = se.sum_total_tt(total_tt_dict, level='stop')
sum_vehicle = se.sum_total_tt(total_tt_dict, level='vehicle')
sum_total = se.sum_total_tt(total_tt_dict, level='total')

# sum splitted into respecitvely waiting & iv time
wt_stops = se.sum_total_tt(waiting_time_dict, level='stop')
ivt_stops = se.sum_total_tt(inveh_time_dict, level='stop')

In [15]:
sum_stops

{1: {1: 136.08739342389654,
  2: 128.3279573251155,
  3: 45.36246447463218,
  4: 25.665591465023105,
  5: 0},
 2: {1: 176.08008745909285,
  2: 170.11136900367882,
  3: 85.55616927172483,
  4: 5.968718455414027,
  5: 0},
 3: {1: 146.2375162494506,
  2: 140.26879779403657,
  3: 41.78102918789819,
  4: 0,
  5: 0},
 4: {1: 303.42000876519046,
  2: 297.4512903097764,
  3: 41.78102918789819,
  4: 0,
  5: 0},
 5: {1: 363.1105968774238,
  2: 357.14187842200977,
  3: 41.78102918789819,
  4: 0,
  5: 0},
 6: {1: 258.45582190044917,
  2: 252.4871034450351,
  3: 41.78102918789819,
  4: 0,
  5: 0},
 7: {1: 490.4505181835214,
  2: 484.4817997281074,
  3: 41.78102918789817,
  4: 0,
  5: 0},
 8: {1: 550.1411062957545,
  2: 544.1723878403404,
  3: 41.781029187898156,
  4: 0,
  5: 0},
 9: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0},
 10: {1: 74.60915087057998, 2: 162.9457755837364, 3: 0, 4: 0, 5: 0},
 11: {1: 128.3279573251155, 2: 112.41046716185333, 3: 0, 4: 0, 5: 0},
 12: {1: 128.3279573251155, 2: 112.4104671618533

In [16]:
sum_vehicle

{1: 335.4434066886674,
 2: 437.71634418991056,
 3: 328.28734323138536,
 4: 642.652328262865,
 5: 762.0335044873318,
 6: 552.7239545333824,
 7: 1016.713347099527,
 8: 1136.0945233239931,
 9: 0,
 10: 237.5549264543164,
 11: 240.73842448696882,
 12: 240.73842448696882,
 13: 215.27044022574927,
 14: 224.82093432370658,
 15: 224.82093432370658,
 16: 192.98595399718224,
 17: 208.9034441604444,
 18: 208.90344416044437,
 19: 178.66701996643218,
 20: 200.95150619499924,
 21: 200.95150619499924,
 22: 188.2175140643895,
 23: 216.86899635826146,
 24: 216.86899635826146,
 25: 197.7680081623468,
 26: 232.78648652152364,
 27: 232.78648652152364,
 28: 207.31850226030411,
 29: 248.70397668478566,
 30: 176.2821192884171,
 31: 74.60915087057998,
 32: 128.3279573251155,
 33: 128.3279573251155,
 34: 74.60915087057998,
 35: 128.32795732511536,
 36: 74.6091508705799}

In [17]:
sum_total

10313.38412159988

In [18]:
occ = se.calc_occupancy_rate(corr_initial, vehicle_capacity)

## Solution visualisation

In [19]:
df_solution = sv.convert_to_dataframe(corr_initial)
df_wt = sv.convert_to_dataframe(wt_stops)
df_ivt = sv.convert_to_dataframe(wt_stops)
df_occ = sv.convert_to_dataframe(occ)

col_names = ['dep_time','abboard_pax1','abboard_pax2','abboard_pax3','sum_wt','sum_ivt','veh_occ']
df_all = pd.concat([df_solution,df_wt, df_ivt, df_occ], axis=1)
df_all.columns = col_names

In [20]:
df_all

,dep_time,abboard_pax1,abboard_pax2,abboard_pax3,sum_wt,sum_ivt,veh_occ
"(1, 1)",1.790718,"[((1, 3), 0, 0), ((1, 3), 0.1989686270407775, ...","[((1, 2), 0, 0)]",None,10.744306,10.744306,0.55
"(1, 2)",7.759436,"[((1, 3), 0, 0), ((1, 3), 0.1989686270407775, ...","[((2, 3), 0, 0), ((2, 3), 0.1989686270407775, ...",None,68.640773,68.640773,1.00
"(1, 3)",13.728155,"[((3, 5), 0, 0)]","[((3, 4), 0, 0)]",None,27.456309,27.456309,0.10
"(1, 4)",19.696873,"[((3, 5), 0, 0)]","[((4, 5), 0, 0)]",None,19.696873,19.696873,0.10
"(1, 5)",25.665591,None,None,None,0.000000,0.000000,0.00
...,...,...,...,...,...,...,...
"(36, 1)",59.889557,"[((1, 3), 58.8947136040701, 0), ((1, 3), 59.09...",None,None,2.984529,2.984529,0.30
"(36, 2)",65.858275,"[((1, 3), 58.8947136040701, 0), ((1, 3), 59.09...",None,None,0.000000,0.000000,0.30
"(36, 3)",71.826994,None,None,None,0.000000,0.000000,0.00
"(36, 4)",NaN,None,None,None,0.000000,0.000000,0.00


In [21]:
df_all.to_excel("Exports/entire_solution.xlsx")